In [3]:
import pandas as pd
import numpy as np
from itertools import groupby  # 用于统计连续交易结果

# ===================== 策略核心配置（修复后）=====================
CONFIG = {
    "widetable_path": r'D:\workspace\xiaoyao\data\widetable.parquet',
    "backtest_path": r'./reverse_strategy_backtest_fixed.csv',
    "index_volatility_window": 30,  # 市场风格识别窗口
    "slip_rate": 0.001,             # 滑点降至0.1%（减少收益侵蚀）
    "max_single_loss_ratio": 0.01,  # 保留单次最大亏损控制（风险底线）
    "continuous_loss_limit": 2,     # 仅保留统计，不实际降仓
    "continuous_profit_limit": 3,
    "buy_down_ratio": 0.02,         # 买入价比例（前5日低点下方2%）
    # 基础选股参数（恢复旧策略宽松条件）
    "auction_ratio_threshold": 0.8,  # 竞价量比≤0.8
    "industry_cold_rank": 0.5,       # 行业热度后50%
    "3d_loss_threshold": -7.0,       # 前3日跌幅≥7%
    # 分市场风格交易参数（修复为高盈亏比配置）
    "market_style_params": {
        "shock": {  # 震荡市（恢复旧策略核心参数）
            "stop_profit": 3.6,      # 止盈3.6%（盈亏比1.8）
            "stop_loss": -2.0,       # 止损-2%
            "trailing_stop_profit": 2.0,  # 保留移动止盈（锁定盈利）
            "single_position_ratio": 0.20  # 恢复20%单票仓位
        },
        "bull": {  # 牛市（保持收益弹性）
            "stop_profit": 4.0,      # 止盈4.0%
            "stop_loss": -2.0,       # 止损2.0%
            "trailing_stop_profit": 2.5,
            "single_position_ratio": 0.20  # 恢复20%仓位
        },
        "bear": {  # 熊市（适度保守，避免过度压缩收益）
            "stop_profit": 3.2,      # 止盈3.2%（高于原2.8%）
            "stop_loss": -1.8,       # 止损1.8%（盈亏比1.78）
            "trailing_stop_profit": 1.5,
            "single_position_ratio": 0.15  # 仓位15%（高于原10%）
        }
    },
    "initial_fund": 100000.0
}

# ===================== 全局变量 =====================
TRADING_DAYS = []  # 有效交易日列表
MARKET_VOLATILITY = {}  # 每日市场波动率
CONTINUOUS_RESULTS = []  # 连续交易结果（仅统计，不用于降仓）


# ===================== 数据加载与预处理 =====================
def load_data():
    global TRADING_DAYS, MARKET_VOLATILITY
    
    # 加载宽表数据
    df = pd.read_parquet(CONFIG["widetable_path"])
    df["date"] = pd.to_datetime(df["date"])
    
    # 保留必要的风险过滤（ST股+停牌股）
    df = df[df["stock_name"].notna()].copy()  # 处理缺失值
    df = df[(df["paused"] == 0.0) & (~df["stock_name"].str.contains("ST", na=False))].copy()
    
    # 1. 计算核心因子（与旧策略一致，确保信号逻辑不变）
    # 竞价量比
    df["auction_vol_5d_mean"] = df.groupby("stock_code")["auc_volume"].transform(
        lambda x: x.rolling(5, min_periods=3).mean().shift(1).replace(0, 0.0001)
    )
    df["auc_volume_ratio"] = df["auc_volume"] / df["auction_vol_5d_mean"]
    # 涨跌幅相关
    df["daily_return"] = (df["close"] / df["pre_close"] - 1) * 100
    df["3d_return"] = df.groupby("stock_code")["daily_return"].transform(
        lambda x: x.rolling(3).sum().shift(1)
    )
    # 价格类因子
    df["ma5"] = df.groupby("stock_code")["close"].transform(lambda x: x.rolling(5).mean())
    df["ma20"] = df.groupby("stock_code")["close"].transform(lambda x: x.rolling(20).mean())
    df["5d_low"] = df.groupby("stock_code")["low"].transform(
        lambda x: x.rolling(5).min().shift(1)
    )
    
    # 2. 行业热度排名（与旧策略一致）
    industry_rank = df.groupby(["date", "sw_l1_industry_name"])["daily_return"].mean().reset_index()
    industry_rank["industry_percentile"] = industry_rank.groupby("date")["daily_return"].rank(pct=True)
    df = df.merge(industry_rank[["date", "sw_l1_industry_name", "industry_percentile"]],
                  on=["date", "sw_l1_industry_name"], how="left")
    
    # 3. 市场波动率计算（用于简化后的风格识别）
    market_daily = df.groupby("date")["daily_return"].agg(["mean", "std"]).reset_index()
    market_daily["market_atr"] = market_daily["mean"].rolling(
        CONFIG["index_volatility_window"], min_periods=15
    ).apply(lambda x: np.mean(np.abs(x - x.shift(1))), raw=False)
    MARKET_VOLATILITY = dict(zip(market_daily["date"], market_daily["market_atr"]))
    
    # 4. 有效交易日列表
    TRADING_DAYS = sorted(df["date"].unique())
    return df


def get_market_style(date):
    """简化市场风格识别：减少熊市误判，保留核心区分"""
    atr = MARKET_VOLATILITY.get(date, 2.5)  # 默认震荡市
    if atr > 3.5:  # 提高牛市阈值，避免小幅波动误判为牛市
        return "bull"
    elif atr < 1.5:  # 降低熊市阈值，避免中度波动误判为熊市
        return "bear"
    else:
        return "shock"  # 多数行情归为震荡市（使用高盈亏比参数）


def get_next_trading_day(current_date):
    """获取下一个有效交易日（保留旧策略稳定逻辑）"""
    idx = TRADING_DAYS.index(current_date)
    if idx + 1 < len(TRADING_DAYS):
        return TRADING_DAYS[idx + 1]
    return None


# ===================== 超跌股票筛选（修复核心：放宽条件）=====================
def select_reverse_stocks(df):
    all_selections = []
    for date in TRADING_DAYS:
        daily_df = df[df["date"] == date].copy()
        if len(daily_df) == 0:
            continue
        
        # 修复：恢复旧策略5个核心筛选条件，移除过严约束
        filter_conditions = (
            # 1. 超跌核心条件（保留旧策略逻辑）
            (daily_df["3d_return"] <= CONFIG["3d_loss_threshold"]) &
            (daily_df["ma5"] < daily_df["ma20"]) &  # 均线空头
            (daily_df["daily_return"].shift(1) >= -1.0) &  # 前1日跌幅收窄
            # 2. 资金冷淡条件（保留旧策略逻辑）
            (daily_df["auc_volume_ratio"] <= CONFIG["auction_ratio_threshold"]) &
            (daily_df["industry_percentile"] <= CONFIG["industry_cold_rank"])  # 非热点行业
            # 已移除的过严条件：收阳、未触涨停、高流通市值、高成交额
        )
        daily_df = daily_df[filter_conditions].copy()
        if len(daily_df) == 0:
            continue
        
        # 按超跌程度排序，选前5只（与旧策略一致）
        daily_df["oversold_score"] = daily_df["3d_return"].rank(ascending=True)
        select_count = min(5, len(daily_df))
        selected = daily_df.nsmallest(select_count, "oversold_score")
        selected["selection_date"] = date
        selected["market_style"] = get_market_style(date)  # 保留风格标记（用于参数匹配）
        
        all_selections.append(selected[["selection_date", "stock_code", "5d_low", "market_style"]])
    return pd.concat(all_selections, ignore_index=True)


# ===================== 策略回测（修复核心：取消过度约束）=====================
def adjust_position_ratio():
    """修复：取消连续亏损降仓，仅保留统计功能"""
    return None  # 始终返回None，不调整仓位，保持基础仓位


def backtest_reverse(selections, df):
    global CONTINUOUS_RESULTS
    # 价格映射（保留盘中触发逻辑，提升收益真实性）
    price_map = df.set_index(["date", "stock_code"])[["open", "close", "low", "high"]].to_dict("index")
    fund = CONFIG["initial_fund"]
    records = []
    
    for _, row in selections.iterrows():
        stock = row["stock_code"]
        t_date = row["selection_date"]
        market_style = row["market_style"]
        style_params = CONFIG["market_style_params"][market_style]
        
        # 1. 买卖日期确定（与旧策略一致，确保周期稳定）
        t1_date = get_next_trading_day(t_date)
        t2_date = get_next_trading_day(t1_date) if t1_date else None
        if not t1_date or not t2_date:
            continue
        
        # 2. T+1日买入逻辑（与旧策略一致，确保成本合理）
        t1_data = price_map.get((t1_date, stock))
        if not t1_data:
            continue
        t1_low = t1_data["low"]
        t1_open = t1_data["open"]
        target_buy_price = row["5d_low"] * (1 - CONFIG["buy_down_ratio"])
        
        # 确定买入价（与旧策略一致）
        if t1_low <= target_buy_price <= t1_open:
            raw_buy_price = target_buy_price
        elif t1_low <= target_buy_price:
            raw_buy_price = t1_low
        else:
            continue
        # 滑点调整（降至0.1%，减少侵蚀）
        buy_price = raw_buy_price * (1 + CONFIG["slip_rate"])
        
        # 3. 仓位计算（恢复20%基础仓位，保留风险控制）
        base_position = style_params["single_position_ratio"]
        adjust_ratio = adjust_position_ratio()
        position_ratio = adjust_ratio if adjust_ratio is not None else base_position
        
        # 单次最大亏损控制（保留风险底线，避免极端亏损）
        max_loss = fund * CONFIG["max_single_loss_ratio"]
        stop_loss_price = buy_price * (1 + style_params["stop_loss"] / 100)
        loss_per_share = buy_price - stop_loss_price
        if loss_per_share <= 0:
            continue
        max_shares_by_risk = int(max_loss // (loss_per_share * 100)) * 100
        max_shares_by_fund = int((fund * position_ratio) // (buy_price * 100)) * 100
        shares = min(max_shares_by_fund, max_shares_by_risk)
        if shares <= 0:
            continue
        
        # 4. T+2日卖出逻辑（保留盘中触发，叠加移动止盈）
        t2_data = price_map.get((t2_date, stock))
        if not t2_data:
            continue
        t2_open = t2_data["open"]
        t2_high = t2_data["high"]
        t2_low = t2_data["low"]
        t2_close = t2_data["close"]
        
        # 关键价格计算（恢复高盈亏比参数）
        take_profit_price = buy_price * (1 + style_params["stop_profit"] / 100)
        stop_loss_price = buy_price * (1 + style_params["stop_loss"] / 100)
        trailing_stop_trigger = buy_price * (1 + style_params["trailing_stop_profit"] / 100)
        trailing_stop_price = buy_price
        
        # 跳空保护（保留风险控制，避免极端跳空）
        if t2_open <= stop_loss_price:
            raw_sell_price = t2_open
            trigger_type = "跳空止损"
        else:
            # 移动止盈（保留，锁定盈利，避免回吐）
            if t2_high >= trailing_stop_trigger:
                if t2_high >= take_profit_price:
                    raw_sell_price = take_profit_price
                    trigger_type = "动态止盈"
                else:
                    if t2_low <= trailing_stop_price:
                        raw_sell_price = trailing_stop_price
                        trigger_type = "移动止损锁定盈利"
                    else:
                        raw_sell_price = t2_close
                        trigger_type = "收盘价卖出（移动保护）"
            else:
                # 常规止盈止损（与旧策略一致）
                if t2_high >= take_profit_price:
                    raw_sell_price = take_profit_price
                    trigger_type = "常规止盈"
                elif t2_low <= stop_loss_price:
                    raw_sell_price = stop_loss_price
                    trigger_type = "常规止损"
                else:
                    raw_sell_price = t2_close
                    trigger_type = "收盘价卖出"
        
        # 卖出滑点（0.1%）
        sell_price = raw_sell_price * (1 - CONFIG["slip_rate"])
        # 收益计算
        return_rate = (sell_price / buy_price - 1) * 100
        profit = (sell_price - buy_price) * shares
        fund += profit
        # 记录交易结果（仅统计，不用于降仓）
        trade_result = "profit" if return_rate > 0 else "loss"
        CONTINUOUS_RESULTS.append(trade_result)
        
        # 详细记录（便于后续分析）
        records.append({
            "stock_code": stock,
            "selection_date": t_date,
            "buy_date": t1_date,
            "sell_date": t2_date,
            "market_style": market_style,
            "buy_price": buy_price,
            "sell_price": sell_price,
            "shares": shares,
            "position_ratio": position_ratio,
            "return_rate": return_rate,
            "trigger_type": trigger_type,
            "trade_result": trade_result,
            "fund_after": fund
        })
    
    # 回测结果统计（与旧策略一致，增加风格维度）
    backtest_df = pd.DataFrame(records)
    valid_returns = backtest_df["return_rate"].dropna()
    total_trades = len(valid_returns)
    if total_trades == 0:
        print("无有效交易记录")
        return backtest_df
    
    # 核心指标计算
    profit_trades = valid_returns[valid_returns > 0]
    loss_trades = valid_returns[valid_returns <= 0]
    win_rate = len(profit_trades) / total_trades * 100
    avg_profit = profit_trades.mean() if len(profit_trades) > 0 else 0
    avg_loss = abs(loss_trades.mean()) if len(loss_trades) > 0 else 0
    profit_loss_ratio = avg_profit / avg_loss if avg_loss != 0 else 0
    total_return = (fund - CONFIG["initial_fund"]) / CONFIG["initial_fund"] * 100
    
    # 分风格统计（保留，便于观察行情适应性）
    style_stats = backtest_df.groupby("market_style").agg({
        "return_rate": ["count", lambda x: len(x[x>0])/len(x)*100 if len(x) else 0],
        "fund_after": lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[0]*100 if len(x) > 1 else 0
    }).round(2)
    style_stats.columns = ["交易次数", "胜率(%)", "阶段收益(%)"]
    
    # 连续交易统计（保留，便于风险监控）
    max_continuous_profit = 0
    max_continuous_loss = 0
    if CONTINUOUS_RESULTS:
        for key, group in groupby(CONTINUOUS_RESULTS):
            group_len = len(list(group))
            if key == "profit":
                max_continuous_profit = max(max_continuous_profit, group_len)
            else:
                max_continuous_loss = max(max_continuous_loss, group_len)
    
    # 结果输出（清晰展示核心指标）
    print("="*80)
    print(f"超跌弱反弹策略回测结果（修复版）")
    print("="*80)
    print(f"初始资金：{CONFIG['initial_fund']:.2f} 元 → 最终资金：{fund:.2f} 元")
    print(f"总收益率：{total_return:.2f}% | 总交易次数：{total_trades}")
    print(f"胜率：{win_rate:.2f}% | 平均盈利：{avg_profit:.2f}% | 平均亏损：{avg_loss:.2f}%")
    print(f"盈亏比：{profit_loss_ratio:.2f} | 最大连续盈利：{max_continuous_profit} 笔")
    print(f"最大连续亏损：{max_continuous_loss} 笔")
    print("\n分市场风格表现：")
    print(style_stats)
    print("="*80)
    
    backtest_df.to_csv(CONFIG["backtest_path"], index=False)
    return backtest_df


# ===================== 策略执行入口 =====================
if __name__ == "__main__":
    # 初始化全局变量
    TRADING_DAYS = []
    MARKET_VOLATILITY = {}
    CONTINUOUS_RESULTS = []
    
    # 执行流程（与旧策略一致，确保稳定）
    print("1. 加载数据并计算因子...")
    df = load_data()
    print(f"   数据时间范围：{df['date'].min().strftime('%Y-%m-%d')} ~ {df['date'].max().strftime('%Y-%m-%d')}")
    
    print("2. 筛选超跌股票...")
    selections = select_reverse_stocks(df)
    print(f"   有效选股信号数：{len(selections)} 个")  # 预期恢复至800-1000个
    
    print("3. 执行策略回测...")
    backtest_result = backtest_reverse(selections, df)
    print(f"4. 回测结果已保存至：{CONFIG['backtest_path']}")

1. 加载数据并计算因子...
   数据时间范围：2023-01-03 ~ 2025-10-24
2. 筛选超跌股票...
   有效选股信号数：2561 个
3. 执行策略回测...
超跌弱反弹策略回测结果（修复版）
初始资金：100000.00 元 → 最终资金：187201.39 元
总收益率：87.20% | 总交易次数：932
胜率：50.32% | 平均盈利：2.98% | 平均亏损：2.04%
盈亏比：1.46 | 最大连续盈利：12 笔
最大连续亏损：12 笔

分市场风格表现：
              交易次数  胜率(%)  阶段收益(%)
market_style                      
bear           540  47.41    87.22
shock          392  54.34    30.84
4. 回测结果已保存至：./reverse_strategy_backtest_fixed.csv
